## 0. Import necessary modules

In [1]:
# Data Manipulation
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Imputation of Missing Data
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [2]:
# Extend number of Pandas df columns to display
pd.set_option('display.max_columns', 60)

In [3]:
%matplotlib inline

## 1. Load data

We load the dataframes whose preliminary analysis was performed in Stage_1, and also the information concerning missing values strategies.

In [4]:
# Local paths to data
proc_data_PATH = './processed_data/'

In [5]:
with open(proc_data_PATH + 'dataframes.pkl', 'rb') as f:
    db_20k_fs, db_10k = pickle.load(f)

In [6]:
with open(proc_data_PATH + 'MV_imputation_dict.pkl', 'rb') as f:
    MV_imputation = pickle.load(f)

In [7]:
db_20k_fs.head(2)

,Price Sensitivity,PhoneType,Email,Tenure,NumberofCampaigns,ProdActive,ProdBought,Birthdate,Socieconomic Status,Province,Right Address,Living Area (m^2),House Price,Income,yearBuilt,House Insurance,Pension Plan,Estimated number of cars,Probability of Second Residence,Credit,Savings,Number of Mobile Phones,Number of Fixed Lines,ADSL,3G Devices,Type of House,PhoneType_cat,Province_cat,Right Address_cat,Type of House_cat,Socieconomic Status_cat,Estim. num. cars_cat,Proba. Sec. Res._cat,Sales,Premium Offered
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,FIXED / LAND PHONE,0,2012,2,0,0,NaN,Medium,MA,Right,68.28964,439134.274215,55407.736314,2005.0,15345.3798,76897.09528,two,Low,35147.05566,0.0,NaN,NaN,NaN,NaN,Urban,0,19.0,0.0,1.0,1.0,2.0,0.0,0,11.12
1,NaN,MOBILE PHONE,0,2011,2,0,1,1961.0,Medium,CA,Right,85.60244,103235.979015,27854.157114,1998.0,3485.5758,18037.38088,None,Low,8133.08226,0.0,NaN,NaN,NaN,NaN,Rustic,1,6.0,0.0,0.0,1.0,0.0,0.0,1,11.12


In [8]:
db_10k.head(2)

,Price Sensitivity,PhoneType,Email,Tenure,NumberofCampaigns,ProdActive,ProdBought,Birthdate,Socieconomic Status,Province,Right Address,Living Area (m^2),House Price,Income,yearBuilt,House Insurance,Pension Plan,Estimated number of cars,Probability of Second Residence,Credit,Savings,Number of Mobile Phones,Number of Fixed Lines,ADSL,3G Devices,Type of House,PhoneType_cat,Province_cat,Right Address_cat,Type of House_cat,Socieconomic Status_cat,Estim. num. cars_cat,Proba. Sec. Res._cat
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,6.0,FIXED / LAND PHONE,0,2011,4,0,0,1962.0,High,OU,Right,60.4474,230139.624195,62842.06971,1961.0,8010.6894,40289.464296,One,Low,18386.272962,56033.07732,2.0,1.0,1.0,0.0,Urban,0,22.0,0.0,1.0,2.0,1.0,0.0
1,NaN,FIXED / LAND PHONE,0,2012,2,0,0,NaN,Medium,VA,Right,78.0148,119859.179895,21367.73331,1955.0,2678.2794,13824.893096,None,Low,6240.219762,0.00000,NaN,NaN,NaN,NaN,Urban,0,31.0,0.0,1.0,1.0,0.0,0.0


## 2. Preprocess data

First we drop those categorical columns where categories are represented by strings (equivalent columns populated by numeric values were already added)

In [9]:
db_20k_fs = db_20k_fs.drop(['PhoneType','Socieconomic Status','Province',
                           'Right Address','Estimated number of cars','Type of House',
                           'Probability of Second Residence'],axis=1)

db_10k = db_10k.drop(['PhoneType','Socieconomic Status','Province',
                           'Right Address','Estimated number of cars','Type of House',
                           'Probability of Second Residence'],axis=1)

### 2.1 Generation of Imputer and One-Hot-Encoding of Categorical Variables

We have 2 datasets: db_20k_fs and db_10k. They share all columns except for 'Sales' and 'Premium Offered', our target and optimization variables, which are not present in the latter.

db_20k_fs will be divided in two: a training set and a test set, so we can validate the performane of our predictive algorithm before we use it for optimization of revenue with the db_10k

The rest of the columns/features have many missing values, which need to be imputed. The imputation process must be strictly based on the training set to avoid data leakage, and then applied to the test set, and also to our optimization dataset.

There is also the issue with categorical features which need to be encoded into binary columns (in our case there is only one such feature: "Province"). When we use methods like pd.get_dummies, or sklearn one-hot-encoder on each dataset separately, there can be mismatches in the number of columns when one of the categories is not present in one of them (which is the case for us).  
In order to bypass this situation, we should apply the enconding step to all our data combined, so the same number of binary columns is generated. Then we can break the combined dataset down into its original components.

In [10]:
df_aux_IMP_DUM = pd.concat([db_20k_fs.drop(['Sales','Premium Offered'],axis=1),db_10k]).reset_index()
df_aux_IMP_DUM.head()

,index,Price Sensitivity,Email,Tenure,NumberofCampaigns,ProdActive,ProdBought,Birthdate,Living Area (m^2),House Price,Income,yearBuilt,House Insurance,Pension Plan,Credit,Savings,Number of Mobile Phones,Number of Fixed Lines,ADSL,3G Devices,PhoneType_cat,Province_cat,Right Address_cat,Type of House_cat,Socieconomic Status_cat,Estim. num. cars_cat,Proba. Sec. Res._cat
0,0,NaN,0,2012,2,0,0,NaN,68.28964,439134.274215,55407.736314,2005.0,15345.3798,76897.09528,35147.05566,0.000,NaN,NaN,NaN,NaN,0,19.0,0.0,1.0,1.0,2.0,0.0
1,1,NaN,0,2011,2,0,1,1961.0,85.60244,103235.979015,27854.157114,1998.0,3485.5758,18037.38088,8133.08226,0.000,NaN,NaN,NaN,NaN,1,6.0,0.0,0.0,1.0,0.0,0.0
2,2,NaN,0,1992,2,0,0,1950.0,170.12964,398515.957515,108942.578514,1974.0,13911.2298,69779.49768,31880.39706,98010.783,3.0,2.0,1.0,0.0,0,25.0,0.0,1.0,1.0,2.0,0.0
3,3,NaN,0,2011,2,0,0,1962.0,158.64584,231849.048015,63173.280114,2007.0,8026.6158,40574.30488,18476.52126,0.000,NaN,NaN,NaN,NaN,1,26.0,0.0,1.0,1.0,1.0,0.0
4,4,NaN,0,1992,2,0,0,1969.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


#### Missing Data Imputer

We will use ColumnTransformer and SimpleImputer modules from sklearn to create a common transformer that takes care of the imputation. We need to train it first (using its .fit() method) using only the training set.  
For that, we firs extract the training dataset (enforcing the "random_state" value so we can then reproduce the same split).

In [11]:
db_20k_train, db_20k_test = train_test_split(db_20k_fs, test_size=0.2, random_state=17, stratify = db_20k_fs.Sales)
X_train = db_20k_train.drop(['Sales','Premium Offered'],axis=1)
print(db_20k_train.shape)
print(db_20k_test.shape)

(16000, 28)
(4000, 28)


We previously defined a dictionary containing the name of each feature and the imputation strategy to be use.
We will now generate one list containing all features name for each imputation strategies, which will be then passed to ColumnTransformer.

In [12]:
drop_ft = []
median_ft = []
mode_ft = []
custom_ft = []
for ft, method in MV_imputation.items():
    if method == 'drop':
        drop_ft.append(ft)
    elif method == 'median':
        median_ft.append(ft)
    elif method == 'mode':
        mode_ft.append(ft)
    elif method == 'custom':
        custom_ft.append(ft)

# We also define a list with those features that have no missing values
no_MV_ft = ['PhoneType_cat','Email','Tenure','NumberofCampaigns','ProdActive','ProdBought']

In [13]:
print(drop_ft)
print(median_ft)
print(mode_ft)
print(custom_ft)

['Number of Mobile Phones', 'Number of Fixed Lines', 'ADSL', '3G Devices', 'Birthdate']
['Living Area (m^2)', 'House Price', 'Income', 'House Insurance', 'Pension Plan', 'Estim. num. cars_cat', 'Proba. Sec. Res._cat', 'Credit', 'Savings', 'yearBuilt']
['Socieconomic Status_cat', 'Province_cat', 'Right Address_cat', 'Type of House_cat']
['Price Sensitivity']


In [14]:
median_imputer = SimpleImputer(strategy='median')
mode_imputer = SimpleImputer(strategy='most_frequent')
data_imputer = ColumnTransformer(
    transformers=[
        ('drp','drop',drop_ft),
        ('med', median_imputer, median_ft),
        ('mod', mode_imputer, mode_ft),
        ('rest','passthrough',no_MV_ft + custom_ft)])

#### Training of imputer

In [15]:
data_imputer.fit(X_train)

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
         transformer_weights=None,
         transformers=[('drp', 'drop', ['Number of Mobile Phones', 'Number of Fixed Lines', 'ADSL', '3G Devices', 'Birthdate']), ('med', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='median', verbose=0), ['Living Area (m^2)', 'House Price', 'Income', 'House Insurance', 'Pensi...ype_cat', 'Email', 'Tenure', 'NumberofCampaigns', 'ProdActive', 'ProdBought', 'Price Sensitivity'])])

#### Application of imputer

In [16]:
aux_IMP_DUM = data_imputer.transform(df_aux_IMP_DUM)

sklearn's Imputer returns an array where the columns are ordered in the same way as the lists of features that were passed to it.  
We now get back our pandas dataframe to keep the feature name information in the columns.

In [17]:
df_aux_IMP_DUM_i = pd.DataFrame(data = aux_IMP_DUM, index = df_aux_IMP_DUM['index'], columns = median_ft + mode_ft + no_MV_ft + custom_ft)
df_aux_IMP_DUM_i.head()

,Living Area (m^2),House Price,Income,House Insurance,Pension Plan,Estim. num. cars_cat,Proba. Sec. Res._cat,Credit,Savings,yearBuilt,Socieconomic Status_cat,Province_cat,Right Address_cat,Type of House_cat,PhoneType_cat,Email,Tenure,NumberofCampaigns,ProdActive,ProdBought,Price Sensitivity
index,,,,,,,,,,,,,,,,,,,,,
0,68.28964,439134.274215,55407.736314,15345.3798,76897.09528,2.0,0.0,35147.05566,0.000,2005.0,1.0,19.0,0.0,1.0,0.0,0.0,2012.0,2.0,0.0,0.0,NaN
1,85.60244,103235.979015,27854.157114,3485.5758,18037.38088,0.0,0.0,8133.08226,0.000,1998.0,1.0,6.0,0.0,0.0,1.0,0.0,2011.0,2.0,0.0,1.0,NaN
2,170.12964,398515.957515,108942.578514,13911.2298,69779.49768,2.0,0.0,31880.39706,98010.783,1974.0,1.0,25.0,0.0,1.0,0.0,0.0,1992.0,2.0,0.0,0.0,NaN
3,158.64584,231849.048015,63173.280114,8026.6158,40574.30488,1.0,0.0,18476.52126,0.000,2007.0,1.0,26.0,0.0,1.0,1.0,0.0,2011.0,2.0,0.0,0.0,NaN
4,118.08404,170171.901315,46198.509714,5854.6278,29799.41848,0.0,0.0,13516.27566,0.000,1980.0,1.0,4.0,0.0,1.0,1.0,0.0,1992.0,2.0,0.0,0.0,NaN


In [18]:
def price_sens_imputer(df_in):
    df = df_in.copy()
    c1 = (df['Socieconomic Status_cat'] == 3) & (df['Price Sensitivity'].isna())
    c2 = (df['Socieconomic Status_cat'] < 3) & (df['Price Sensitivity'].isna())
    df.loc[c1, 'Price Sensitivity'] = 3
    df.loc[c2, 'Price Sensitivity'] = 5
    return df

In [21]:
df_aux_IMP_DUM_i = price_sens_imputer(df_aux_IMP_DUM_i)

In [22]:
df_aux_IMP_DUM_i.head()

,Living Area (m^2),House Price,Income,House Insurance,Pension Plan,Estim. num. cars_cat,Proba. Sec. Res._cat,Credit,Savings,yearBuilt,Socieconomic Status_cat,Province_cat,Right Address_cat,Type of House_cat,PhoneType_cat,Email,Tenure,NumberofCampaigns,ProdActive,ProdBought,Price Sensitivity
index,,,,,,,,,,,,,,,,,,,,,
0,68.28964,439134.274215,55407.736314,15345.3798,76897.09528,2.0,0.0,35147.05566,0.000,2005.0,1.0,19.0,0.0,1.0,0.0,0.0,2012.0,2.0,0.0,0.0,5.0
1,85.60244,103235.979015,27854.157114,3485.5758,18037.38088,0.0,0.0,8133.08226,0.000,1998.0,1.0,6.0,0.0,0.0,1.0,0.0,2011.0,2.0,0.0,1.0,5.0
2,170.12964,398515.957515,108942.578514,13911.2298,69779.49768,2.0,0.0,31880.39706,98010.783,1974.0,1.0,25.0,0.0,1.0,0.0,0.0,1992.0,2.0,0.0,0.0,5.0
3,158.64584,231849.048015,63173.280114,8026.6158,40574.30488,1.0,0.0,18476.52126,0.000,2007.0,1.0,26.0,0.0,1.0,1.0,0.0,2011.0,2.0,0.0,0.0,5.0
4,118.08404,170171.901315,46198.509714,5854.6278,29799.41848,0.0,0.0,13516.27566,0.000,1980.0,1.0,4.0,0.0,1.0,1.0,0.0,1992.0,2.0,0.0,0.0,5.0


We now save these dataframes with their missing values dealt with, but before we binarize 'Province', since we will use it for exploratory data analysis, and the dummy binary variables are not adequate for that task.

In [23]:
db_20k_imp_nd = df_aux_IMP_DUM_i.iloc[:20000,:].join(db_20k_fs[['Sales','Premium Offered']])
db_10k_imp_nd = df_aux_IMP_DUM_i.iloc[20000:,:]

with open(proc_data_PATH + 'proc_dataframes_no_dummies.pkl', 'wb') as f:
    pickle.dump([db_20k_imp_nd,db_10k_imp_nd], f)

***

#### One-Hot-Encoding of Province feature

In [24]:
df_aux_IMP_DUM_id = pd.get_dummies(df_aux_IMP_DUM_i, columns=['Province_cat'], drop_first=True)
df_aux_IMP_DUM_id.head()

,Living Area (m^2),House Price,Income,House Insurance,Pension Plan,Estim. num. cars_cat,Proba. Sec. Res._cat,Credit,Savings,yearBuilt,Socieconomic Status_cat,Right Address_cat,Type of House_cat,PhoneType_cat,Email,Tenure,NumberofCampaigns,ProdActive,ProdBought,Price Sensitivity,Province_cat_1.0,Province_cat_2.0,Province_cat_3.0,Province_cat_4.0,Province_cat_5.0,Province_cat_6.0,Province_cat_7.0,Province_cat_8.0,Province_cat_9.0,Province_cat_10.0,Province_cat_11.0,Province_cat_12.0,Province_cat_13.0,Province_cat_14.0,Province_cat_15.0,Province_cat_16.0,Province_cat_17.0,Province_cat_18.0,Province_cat_19.0,Province_cat_20.0,Province_cat_21.0,Province_cat_22.0,Province_cat_23.0,Province_cat_24.0,Province_cat_25.0,Province_cat_26.0,Province_cat_27.0,Province_cat_28.0,Province_cat_29.0,Province_cat_30.0,Province_cat_31.0,Province_cat_32.0,Province_cat_33.0
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,68.28964,439134.274215,55407.736314,15345.3798,76897.09528,2.0,0.0,35147.05566,0.000,2005.0,1.0,0.0,1.0,0.0,0.0,2012.0,2.0,0.0,0.0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,85.60244,103235.979015,27854.157114,3485.5758,18037.38088,0.0,0.0,8133.08226,0.000,1998.0,1.0,0.0,0.0,1.0,0.0,2011.0,2.0,0.0,1.0,5.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,170.12964,398515.957515,108942.578514,13911.2298,69779.49768,2.0,0.0,31880.39706,98010.783,1974.0,1.0,0.0,1.0,0.0,0.0,1992.0,2.0,0.0,0.0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,158.64584,231849.048015,63173.280114,8026.6158,40574.30488,1.0,0.0,18476.52126,0.000,2007.0,1.0,0.0,1.0,1.0,0.0,2011.0,2.0,0.0,0.0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,118.08404,170171.901315,46198.509714,5854.6278,29799.41848,0.0,0.0,13516.27566,0.000,1980.0,1.0,0.0,1.0,1.0,0.0,1992.0,2.0,0.0,0.0,5.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Original datasets recovery

We finally put the imputed and encoded data into separated datasets, respecting the original distribution of the samples.

In [25]:
db_20k_final = df_aux_IMP_DUM_id.iloc[:20000,:].join(db_20k_fs[['Sales','Premium Offered']])
db_20k_final.tail()

,Living Area (m^2),House Price,Income,House Insurance,Pension Plan,Estim. num. cars_cat,Proba. Sec. Res._cat,Credit,Savings,yearBuilt,Socieconomic Status_cat,Right Address_cat,Type of House_cat,PhoneType_cat,Email,Tenure,NumberofCampaigns,ProdActive,ProdBought,Price Sensitivity,Province_cat_1.0,Province_cat_2.0,Province_cat_3.0,Province_cat_4.0,Province_cat_5.0,Province_cat_6.0,Province_cat_7.0,Province_cat_8.0,Province_cat_9.0,Province_cat_10.0,Province_cat_11.0,Province_cat_12.0,Province_cat_13.0,Province_cat_14.0,Province_cat_15.0,Province_cat_16.0,Province_cat_17.0,Province_cat_18.0,Province_cat_19.0,Province_cat_20.0,Province_cat_21.0,Province_cat_22.0,Province_cat_23.0,Province_cat_24.0,Province_cat_25.0,Province_cat_26.0,Province_cat_27.0,Province_cat_28.0,Province_cat_29.0,Province_cat_30.0,Province_cat_31.0,Province_cat_32.0,Province_cat_33.0,Sales,Premium Offered
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
19995,95.37104,181411.869015,49322.465514,5225.8878,31736.20088,0.0,0.0,14420.22726,44352.684,1969.0,0.0,0.0,0.0,0.0,0.0,2013.0,2.0,0.0,0.0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.5
19996,96.42964,124193.568915,33609.433914,2453.1678,21709.78488,0.0,0.0,9818.55126,0.000,1980.0,0.0,0.0,1.0,0.0,0.0,2013.0,2.0,0.0,0.0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.5
19997,1009.63964,189604.431315,51572.258514,6535.0458,33171.76968,0.0,0.0,15079.08906,46377.495,1984.0,0.0,0.0,1.0,0.0,0.0,2009.0,4.0,0.0,1.0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.5
19998,414.67964,384516.228915,105098.041314,13416.9318,67326.33288,2.0,0.0,30754.50426,94550.706,1975.0,0.0,0.0,1.0,0.0,0.0,2012.0,4.0,0.0,0.0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,14.5
19999,57.75724,153225.656415,38290.674114,4827.4218,24696.85928,0.0,0.0,11189.48466,0.000,1910.0,3.0,0.0,1.0,1.0,0.0,2012.0,2.0,0.0,0.0,4.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,14.5


In [26]:
db_10k_final = df_aux_IMP_DUM_id.iloc[20000:,:]
db_10k_final.tail()

,Living Area (m^2),House Price,Income,House Insurance,Pension Plan,Estim. num. cars_cat,Proba. Sec. Res._cat,Credit,Savings,yearBuilt,Socieconomic Status_cat,Right Address_cat,Type of House_cat,PhoneType_cat,Email,Tenure,NumberofCampaigns,ProdActive,ProdBought,Price Sensitivity,Province_cat_1.0,Province_cat_2.0,Province_cat_3.0,Province_cat_4.0,Province_cat_5.0,Province_cat_6.0,Province_cat_7.0,Province_cat_8.0,Province_cat_9.0,Province_cat_10.0,Province_cat_11.0,Province_cat_12.0,Province_cat_13.0,Province_cat_14.0,Province_cat_15.0,Province_cat_16.0,Province_cat_17.0,Province_cat_18.0,Province_cat_19.0,Province_cat_20.0,Province_cat_21.0,Province_cat_22.0,Province_cat_23.0,Province_cat_24.0,Province_cat_25.0,Province_cat_26.0,Province_cat_27.0,Province_cat_28.0,Province_cat_29.0,Province_cat_30.0,Province_cat_31.0,Province_cat_32.0,Province_cat_33.0
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9995,102.96560,190645.033995,51996.254310,6616.2294,33368.793096,0.0,0.0,15209.994762,46271.82132,1976.0,1.0,0.0,1.0,1.0,0.0,2012.0,2.0,0.0,0.0,5.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9996,108.36580,129927.848895,35322.384510,4472.4474,22729.300296,0.0,0.0,10326.943962,0.00000,1995.0,0.0,0.0,0.0,0.0,0.0,2012.0,4.0,0.0,0.0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
9997,118.08404,170171.901315,46198.509714,5854.6278,29799.418480,0.0,0.0,13516.275660,0.00000,1980.0,1.0,0.0,1.0,1.0,0.0,2012.0,2.0,0.0,0.0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9998,529.03200,82277.223795,22236.780510,2790.0054,14379.438696,0.0,0.0,6494.731362,51849.99432,1931.0,0.0,0.0,1.0,1.0,0.0,2012.0,2.0,0.0,0.0,5.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9999,118.08404,170171.901315,46198.509714,5854.6278,29799.418480,0.0,0.0,13516.275660,0.00000,1980.0,1.0,1.0,1.0,1.0,0.0,2011.0,4.0,0.0,0.0,5.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


We save our two datasets with no missing values and proper encoding of categorical features so they can be used for training predictive models later.

In [27]:
with open(proc_data_PATH + 'proc_dataframes.pkl', 'wb') as f:
    pickle.dump([db_20k_final,db_10k_final], f)